In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.backend.tensorflow_backend import set_session
config = tf.compat.v1.ConfigProto()
# dynamically grow GPU memory 
config.gpu_options.allow_growth = True 
set_session(tf.compat.v1.Session(config=config))

import re,string,unicodedata
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.pipeline import Pipeline
from nltk import pos_tag
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

df = pd.read_csv("fake_job_postings.csv")
df.fillna(" ",inplace = True)
df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

del df['title']
del df['location']
del df['department']
del df['company_profile']
del df['description']
del df['requirements']
del df['benefits']
del df['employment_type']
del df['required_experience']
del df['required_education']
del df['industry']
del df['function']

print(df.head())

print("\n\nTotal # of job descriptions:")
print(df.shape[0])
print("\n\nPercentage of fraudulent job descriptions:")
print(df[df.fraudulent == 1].shape[0]/(df[df.fraudulent == 0].shape[0]+df[df.fraudulent == 1].shape[0])*100)


Using TensorFlow backend.


   job_id salary_range  telecommuting  has_company_logo  has_questions  \
0       1                           0                 1              0   
1       2                           0                 1              0   
2       3                           0                 1              0   
3       4                           0                 1              0   
4       5                           0                 1              1   

   fraudulent                                               text  
0           0  Marketing Intern US, NY, New York Marketing We...  
1           0  Customer Service - Cloud Video Production NZ, ...  
2           0  Commissioning Machinery Assistant (CMA) US, IA...  
3           0  Account Executive - Washington DC US, DC, Wash...  
4           0  Bill Review Manager US, FL, Fort Worth   SpotS...  


Total # of job descriptions:
17880


Percentage of fraudulent job descriptions:
4.8434004474272925


In [ ]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            pos = pos_tag([i.strip()])
            word = lemmatizer.lemmatize(i.strip(),get_simple_pos(pos[0][1]))
            final_text.append(word.lower())
    return " ".join(final_text)     

df.text = df.text.apply(lemmatize_words)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.text, df.fraudulent, test_size=0.3, random_state=2020)

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

In [ ]:
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(x_train)
#transformed test reviews
cv_test_reviews=cv.transform(x_test)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

In [ ]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tv.transform(x_test)

tv_train_reviews

In [ ]:
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = tv_train_reviews.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()

model.fit(tv_train_reviews,y_train , epochs = 5, batch_size=100)

In [ ]:
accuracy_score(model.predict(cv_test_reviews.todense()),y_test)